In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

sys.path.append("/end/home/rh2310/morpho_repo")
import turing as tu
from turing.utils import *
import turing.loss_functions as tu_loss

GPU = False
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"


# Load the dataset

In [2]:
layers = [3, 64, 64, 64, 64, 2]

# Load Data
import os
data_path = os.path.abspath("turing.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("turing_t.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
T = t_star.shape[0]    
    
L = 50
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size
x_domain = L*np.linspace(0,1,x_size)
y_domain = L*np.linspace(0,1,y_size)

X,Y = np.meshgrid(x_domain, y_domain, sparse=False, indexing='ij')
XX = np.tile(X.flatten(), T) # N x T
YY = np.tile(Y.flatten(), T) # N x T
TT = np.repeat(t_star[-T:], N) # T x N

AA = np.einsum('ijk->kij', data[0, :, :, -T:]).flatten() # N x T
SS = np.einsum('ijk->kij', data[1, :, :, -T:]).flatten() # N x T


boundary_x_LB = np.concatenate((x_domain, 
                                np.repeat(x_domain[0], y_size)))
boundary_x_TR = np.concatenate((x_domain, 
                                np.repeat(x_domain[-1], y_size))) 

boundary_y_LB = np.concatenate((np.repeat(y_domain[0], x_size),
                                y_domain))
boundary_y_TR = np.concatenate((np.repeat(y_domain[-1], x_size),
                                y_domain)) 

boundary_XX_LB = np.tile(boundary_x_LB.flatten(), T)[:, np.newaxis] # (x_size + y_size) x T, 1
boundary_XX_TR = np.tile(boundary_x_TR.flatten(), T)[:, np.newaxis] # (x_size + y_size) x T, 1
boundary_YY_LB = np.tile(boundary_y_LB.flatten(), T)[:, np.newaxis] # (x_size + y_size) x T, 1
boundary_YY_TR = np.tile(boundary_y_TR.flatten(), T)[:, np.newaxis] # (x_size + y_size) x T, 1
boundary_TT = np.repeat(t_star[-T:], (x_size + y_size))[:, np.newaxis] # T x (x_size + y_size), 1


def create_dataset(training_data_size =  T*16,
                   pde_data_size =  (T*N)//(32),
                   boundary_data_size = ((x_size + y_size)*T)//(8),
                   with_boundary = True,
                   signal_to_noise = 0):
    
    ##########################################
    # Including noise
    if signal_to_noise > 0:
        signal_amp_a = (np.max(AA)-np.min(AA))/2.0
        signal_amp_s = (np.max(SS)-np.min(SS))/2.0  
        sigma_a =  signal_amp_a*signal_to_noise
        sigma_s =  signal_amp_s*signal_to_noise
    # Observed data
    idx_data = np.random.choice(N*T, training_data_size, replace=False)
    # PDE colocations
    idx_pde = np.random.choice(N*T, pde_data_size, replace=False)
    # Periodic boundary condition
    idx_boundary = np.random.choice((x_size + y_size)*T, boundary_data_size, replace=False)
    
    ret = {'obs_input': np.c_[XX[idx_data], YY[idx_data],TT[idx_data]],
           'obs_output': np.c_[AA[idx_data], SS[idx_data]],
           'pde':   np.c_[XX[idx_pde], YY[idx_pde], TT[idx_pde]]}
    
    if signal_to_noise > 0:        
        ret['obs_output'][0] += sigma_a * np.random.randn(len(idx_data), a.shape[1])
        ret['obs_output'][1] += sigma_s * np.random.randn(len(idx_data), s.shape[1])
    
    if with_boundary:
        ret = {**ret,
               **{'boundary_LB': np.c_[boundary_XX_LB[idx_boundary], 
                                       boundary_YY_LB[idx_boundary],
                                       boundary_TT[idx_boundary]],
                  'boundary_TR': np.c_[boundary_XX_TR[idx_boundary],
                                       boundary_YY_TR[idx_boundary],
                                       boundary_TT[idx_boundary]]
                 }
              }
    return ret



# Create a TINN model

In [3]:
model_params_1 = {'training_data_size': T*16,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(**model_params_1)
lb, ub = lower_upper_bounds([np.c_[XX, YY, TT]])

In [4]:
pinn = tu.NN(layers, lb, ub)
loss_stack = [
          tu_loss.Observations(pinn,
                               inputs_obs = dataset['obs_input'],
                               outputs_obs = dataset['obs_output']),
          tu_loss.Periodic_boundary(pinn,
                                   inputs_LB_boundary = dataset['boundary_LB'], 
                                   inputs_TR_boundary = dataset['boundary_TR']),
          tu_loss.ASDM(pinn,
                       inputs_pde = dataset['pde'])
            ]

2022-06-14 16:38:17.984617: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-14 16:38:17.984686: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ld-rendres10
2022-06-14 16:38:17.984717: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ld-rendres10
2022-06-14 16:38:17.984797: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 440.33.1
2022-06-14 16:38:17.985219: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 440.33.1
2022-06-14 16:38:17.985232: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 440.33.1


In [5]:
model = tu.TINN(pinn, loss_stack)

In [6]:
arr_total_loss, arr_losses = model.train(10,batch_size=dataset['obs_input'].shape[0]//8, print_iter=4)

Epoch: 0, loss:35782.16
Loss_observations:21290.84117414 
Loss_Periodic_Boundary:148.35666737 
Loss_ASDM:14342.96523517 sigma_a:0:0.00550088, sigma_s:0:0.99212036, mu_a:0:0.99337005, rho_a:0:0.99909805, rho_s:0:1.00617647, kappa_a:0:0.99933922
Time:8.54

Epoch: 4, loss:4136.42
Loss_observations:2277.97320737 
Loss_Periodic_Boundary:9.35589391 
Loss_ASDM:1849.09022772 sigma_a:0:0.03542401, sigma_s:0:0.97255828, mu_a:0:0.96067245, rho_a:0:1.03042694, rho_s:0:1.01962660, kappa_a:0:0.98359472
Time:10.68

Epoch: 8, loss:2955.60
Loss_observations:2620.43305905 
Loss_Periodic_Boundary:4.43195952 
Loss_ASDM:330.73293795 sigma_a:0:0.05867701, sigma_s:0:0.96503302, mu_a:0:0.93148148, rho_a:0:1.06177817, rho_s:0:1.03645190, kappa_a:0:0.95735996
Time:10.58

